# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.125163e+02     1.683444e+00
 * time: 0.07442092895507812
     1     1.060166e+01     1.000549e+00
 * time: 1.4081988334655762
     2    -1.175219e+01     1.139735e+00
 * time: 1.520354986190796
     3    -3.379286e+01     7.842161e-01
 * time: 1.6302669048309326
     4    -4.714633e+01     5.825166e-01
 * time: 1.7564399242401123
     5    -5.683189e+01     2.328308e-01
 * time: 1.8736579418182373
     6    -5.968533e+01     1.874247e-01
 * time: 1.9463839530944824
     7    -6.088317e+01     8.674795e-02
 * time: 2.021021842956543
     8    -6.141235e+01     4.472139e-02
 * time: 2.1063590049743652
     9    -6.169802e+01     3.852340e-02
 * time: 2.1841468811035156
    10    -6.185809e+01     4.134820e-02
 * time: 2.2580888271331787
    11    -6.199476e+01     2.409734e-02
 * time: 2.342028856277466
    12    -6.206872e+01     1.660792e-02
 * time: 2.4303858280181885
    13    -6.211469e+01     1.310387e-02
 * time: 2.507253885269

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671069
    AtomicLocal         -18.8557672
    AtomicNonlocal      14.8522639
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336820

    total               -62.261666462606
